In this step we are filtering the tweets of the Key Opinion Leaders, that we retrived in the first step. 


In [1]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [2]:
import pandas as pd
### note that tweets_table_filtered_file_name file needs to be in xlsx format
def simple_preprocess_on_tweets_table_filtered (tweets_table_filtered_file_name = "tweets_table_filtered.xlsx"):
    tweets_table_filtered = pd.read_excel(tweets_table_filtered_file_name)
    tweets_table_filtered['conversation_id'] = tweets_table_filtered['conv_id_new'].apply(lambda x: x.replace("conv_id: ", ""))
    tweets_table_filtered = tweets_table_filtered.add_prefix('KOL_Tweet_')

    return tweets_table_filtered



In [3]:
### reading the filtered tweets table
tweets_table_filtered = simple_preprocess_on_tweets_table_filtered("tweets_table_filtered_final_by_rawwaa.xlsx")
print("There are",tweets_table_filtered.shape[0], "Filtered tweets")
display(tweets_table_filtered.head())


There are 1513 Filtered tweets


,KOL_Tweet_Unnamed: 0,KOL_Tweet_Unnamed: 0.1,KOL_Tweet_Unnamed: 0.1.1,KOL_Tweet_author_id,KOL_Tweet_author_id_new,KOL_Tweet_conv_id_new,KOL_Tweet_conversation_id,KOL_Tweet_created_at,KOL_Tweet_entities.mentions,KOL_Tweet_id,...,KOL_Tweet_Name,KOL_Tweet_Born_in,KOL_Tweet_twitter_user_name,KOL_Tweet_Role,KOL_Tweet_Place,KOL_Tweet_clean_user_name,KOL_Tweet_score_key_words,KOL_Tweet_score_KOP,KOL_Tweet_total_score,KOL_Tweet_language
0,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,David DAVIS,England,@DavidDavisMP,Secretary of State for Exiting the,European Union UK Government,DavidDavisMP,4,1,25,en
1,1,24792,451,155507136,author_id: 155507136,conv_id: 953958453554401283,953958453554401283,2018-01-18T11:53:13.000Z,"[{'start': 28, 'end': 41, 'username': 'GOettin...",9.539585e+17,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,2,1,20,nl
2,2,3894,26,3131144855,author_id: 3131144855,conv_id: 1069285577869205506,1069285577869205506,2018-12-02T17:47:45.000Z,NaN,1.069287e+18,...,Boris JOHNSON,England,@BorisJohnson,Foreign Secretary,UK Government,BorisJohnson,3,1,20,en
3,3,7496,22,540274837,author_id: 540274837,conv_id: 821274533080551424,821274533080551424,2017-01-17T08:39:25.000Z,NaN,8.212757e+17,...,Frances O'GRADY,England,@FrancesOGrady,General Secretary,Trades Union Congress,FrancesOGrady,2,1,20,en
4,4,24385,44,155507136,author_id: 155507136,conv_id: 1066662602443538432,1066662602443538432,2018-11-25T11:59:16.000Z,NaN,1.066663e+18,...,Mark RUTTE,Netherlands,@MinPres,Dutch Government,Prime Minister,MinPres,3,1,20,nl


In [18]:
#top_n_tweets = list(tweets_table_filtered.head()['KOL_Tweet_conversation_id'])
#top_n_tweets.append("702594185942188033")
#print(top_n_tweets)

['740159086667411456', '953958453554401283', '1069285577869205506', '821274533080551424', '1066662602443538432', '702594185942188033']


In [4]:
### saving in a variable all_conversation_ids all the conversation ids of the filtered tweets
all_conversation_ids = set(tweets_table_filtered['KOL_Tweet_conversation_id'])
#sanity check:
print("Number of conversation ids we wish to get their comments:", len(all_conversation_ids))

Number of conversation ids we wish to get their comments: 1356


## Comments

In the following chunk we will check - wheter we already have a csv file for a given conversation id that is in the list,
If it exists there, then the code in the chunk will remove that conversation id from the final list we are passing the reriving comments function.
This is important as if we will change the filter criterions and get more / less tweets, we don't want to retrieve the comments of conversation ids we already have!

In [5]:
all_conversation_ids = all_conversation_ids #top_n_tweets #[]
max_results = 100
limit_amount_of_returned_comments = 1000000
start_time = "2014-01-01T00:00:00Z"
dir_tree_name = "Step 3 Comments"
##############################################################################################3
#### we don't want to get comments for conversation ids that we already searched for their comments!
# thus we first need to see which of the conversation_ids that are inserted in the list are **not** in the files we already have:
from os import listdir
from os.path import isfile, join

import os.path
try:
    os.mkdir(dir_tree_name)
    print("creating directory", dir_tree_name, "to insert all the tables of all the comments of all the given conversation ids")
except:
    print("The dir", dir_tree_name ,"already exist")

print()
conv_ids_we_have_data = [f.split("_")[-1] for f in listdir(dir_tree_name) if isfile(join(dir_tree_name, f, f.split("_")[-1] + "_comments.csv"))]

conv_ids_we_dont_have_data = list(set(all_conversation_ids) - set(conv_ids_we_have_data))

if len(conv_ids_we_dont_have_data) > 20:
    print("There are", len(conv_ids_we_dont_have_data), "Conversation ids to search their comments; see here the first 20:")
    print(conv_ids_we_dont_have_data[0:20])
else:
    print("Conversation ids to search their comments:",conv_ids_we_dont_have_data)
    print("-----------------------------------------------")
print()
if len(conv_ids_we_have_data) > 20:
    print("There are", len(conv_ids_we_have_data), "Conversation ids we already have data on --> here are the first 20:")
    print(conv_ids_we_have_data[0:20])
else : print("Conversation ids we already have data on:",list(set(all_conversation_ids).intersection(conv_ids_we_have_data)))


The dir Step 3 Comments already exist

There are 56 Conversation ids to search their comments; see here the first 20:
['441512178651512832', '695202145465974784', '467029001996083204', '667623488992223232', '463707158790815744', '524273763425464320', '460825769506508800', '467237659975622656', '522395811884584960', '509672197460590593', '464413961493155840', '466520871457546240', '938036991064772609', '425971431928000512', '460802868728299520', '540101456444268544', '468840479992578048', '486453711661985792', '425566205097676800', '656132052627013633']

There are 1303 Conversation ids we already have data on --> here are the first 20:
['1001060267860660224', '1005121567502864384', '1005751929417490432', '1006165970338107392', '1006508055700402181', '1007194366140612608', '1008280586790764544', '1009534238058864641', '1010085021229645825', '1010085259302629378', '1010160817852043264', '1010534193804271616', '1010959223772106753', '1011488843021869057', '1011500804560257024', '1011501121

In [15]:
### This is the function that enables retirving the comments of the conversation ids
# note that we inserted the function the variable 'conv_ids_we_dont_have_data' so we will only search comments of conversation ids that
# we don't have (that we haven't searched for their commetns so far) 
path_for_table_dict = my_twitter_crawler.return_comments_by_tweet_ids(
                            conversation_ids = conv_ids_we_dont_have_data,
                            query = "",
                            start_time = start_time,
                            end_time = "today",
                            max_results = max_results, evaluate_last_token = False,
                            limit_amount_of_returned_comments = limit_amount_of_returned_comments,
                            verbose = False, dir_tree_name = dir_tree_name)

Bringing comments of 587941845780553728
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_587941845780553728 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_587941845780553728\log_comments_for_conversation_id_587941845780553728 to insert all the logs of the comments for the tweet id -  587941845780553728
no more comments from this conversation id
Total amount of collected comments =  3
3
---------------------------------------------------------------
Bringing comments of 876344395548766208
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_876344395548766208 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_876344395548766208\log_comments_for_conversation_id_876344395548766208 to insert all the logs of the comments for the tweet id -  876344395548766208
1 Got from twitter 99 comments, and there are more 

In [14]:
#TwitterCrawler.get_url_by_tweet_id(tweet_id = "1072239308554362881")

In [9]:
# for root, dirs, files in os.walk(directory):
#     for filename in files:
#         if not filename.endswith('csv'): continue
#         file_path = os.path.join(root, filename)
#         #print(file_path)
#         try:
#             comments_table =  pd.read_csv(file_path)
#         except:
#             print(file_path)
            



NameError: name 'directory' is not defined

,author_id,author_id_new,conv_id_new,conversation_id,created_at,entities.mentions,id,id_new,in_reply_to_user_id,lang,...,users.created_at,users.description,users.id,users.name,users.public_metrics.followers_count,users.public_metrics.following_count,users.public_metrics.listed_count,users.public_metrics.tweet_count,users.username,users.verified
0,501139438,author_id: 501139438,coversation_id: 1015337051355480064,1015337051355480064,2019-04-06T08:36:21.000Z,"[{'start': 0, 'end': 11, 'username': 'rich_may...",1114446734040674304,id: 1114446734040674304,51833762,en,...,2012-02-23T20:52:02.000Z,"Proud European. Socialist. Tell me, how #Brexi...",501139438,The Night King 🇬🇧🇪🇺 #JohnsonOut,3906,5000,130,239703,Iceman_cometh1,False
1,718930349,author_id: 718930349,coversation_id: 1015337051355480064,1015337051355480064,2018-11-27T20:39:22.000Z,"[{'start': 0, 'end': 11, 'username': 'MrFranci...",1067518263192707072,id: 1067518263192707072,718930349,en,...,2013-10-17T09:47:17.000Z,🐻Abhors the left wingers and socialists hijack...,718930349,J 🍊🌸,4893,4327,8,283550,Biedaboo,False
2,718930349,author_id: 718930349,coversation_id: 1015337051355480064,1015337051355480064,2018-11-27T20:37:30.000Z,"[{'start': 0, 'end': 11, 'username': 'MrFranci...",1067517794533679104,id: 1067517794533679104,1009747580719419392,en,...,2013-10-17T09:47:17.000Z,🐻Abhors the left wingers and socialists hijack...,718930349,J 🍊🌸,4893,4327,8,283550,Biedaboo,False
3,715007792,author_id: 715007792,coversation_id: 1015337051355480064,1015337051355480064,2018-11-27T20:38:29.000Z,"[{'start': 0, 'end': 9, 'username': 'Biedaboo'...",1067518043989975040,id: 1067518043989975040,718930349,en,...,2012-07-24T22:09:45.000Z,NaN,715007792,J_Hopkins,9,84,0,149,JHopkinsSite,False
4,23978110,author_id: 23978110,coversation_id: 1015337051355480064,1015337051355480064,2018-11-24T22:39:21.000Z,"[{'start': 0, 'end': 11, 'username': 'MrFranci...",1066461297619939328,id: 1066461297619939328,1009747580719419392,en,...,2009-03-12T16:29:48.000Z,"Politics, education, h2g2, history, and CCFC. ...",23978110,Beamish Red,1277,930,14,16183,Disorganised1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,241533433,author_id: 241533433,coversation_id: 1015337051355480064,1015337051355480064,2018-07-06T20:50:40.000Z,"[{'start': 0, 'end': 12, 'username': 'theresa_...",1015337255437832192,id: 1015337255437832192,747807250819981312,en,...,2011-01-22T14:02:36.000Z,NaN,241533433,Chesterlad,1577,2441,15,10171,chesterlad1975,False
3004,361386352,author_id: 361386352,coversation_id: 1015337051355480064,1015337051355480064,2018-07-06T20:50:23.000Z,"[{'start': 0, 'end': 12, 'username': 'theresa_...",1015337184243650560,id: 1015337184243650560,747807250819981312,en,...,2011-08-24T18:16:53.000Z,Arsenal fan. A WWII aviation follower and all ...,361386352,Duckford AFC,1981,3093,50,90788,duckford10,False
3005,133346891,author_id: 133346891,coversation_id: 1015337051355480064,1015337051355480064,2018-07-06T20:50:20.000Z,"[{'start': 0, 'end': 12, 'username': 'theresa_...",1015337172864585728,id: 1015337172864585728,747807250819981312,en,...,2010-04-15T15:20:39.000Z,NaN,133346891,Nick Hoadley,205,443,5,21985,Nick_Hoadley,False
3006,78862105,author_id: 78862105,coversation_id: 1015337051355480064,1015337051355480064,2018-07-06T20:50:20.000Z,"[{'start': 0, 'end': 12, 'username': 'theresa_...",1015337171954302982,id: 1015337171954302982,747807250819981312,und,...,2009-10-01T09:58:30.000Z,NaN,78862105,James Robert 🇺🇦🇭🇰,80,823,3,8940,Jiminho_,False


In [10]:
import os

cols_list = ["author_id", "author_id_new", "conv_id_new", "conversation_id",
"created_at", "entities.mentions",	"id", "id_new",	"in_reply_to_user_id",	"lang",
"public_metrics.like_count","public_metrics.quote_count","public_metrics.reply_count","public_metrics.retweet_count",
"referenced_tweets",	"reply_settings",	"source",	"text",	"users.created_at",	"users.description",	"users.id",
"users.name",	"users.public_metrics.followers_count",	"users.public_metrics.following_count",	"users.public_metrics.listed_count",	"users.public_metrics.tweet_count",
"users.username",	"users.verified"]

directory = 'Step 3 Comments'

all_comments_df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if not filename.endswith('csv'): continue
        file_path = os.path.join(root, filename)
        comments_table =  pd.read_csv(file_path, usecols=cols_list)
        all_comments_df_list.append(comments_table)

all_comments_df = pd.concat(all_comments_df_list)
all_comments_df['conversation_id'] = all_comments_df['conversation_id'].apply(lambda x : str(x))
all_comments_df = all_comments_df.add_prefix('Comment_')

In [13]:

all_comments_df

,Comment_author_id,Comment_author_id_new,Comment_conv_id_new,Comment_conversation_id,Comment_created_at,Comment_entities.mentions,Comment_id,Comment_id_new,Comment_in_reply_to_user_id,Comment_lang,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,2427867374,author_id: 2427867374,coversation_id: 1001060267860660224,1001060267860660224,2018-08-06T13:55:30.000Z,"[{'start': 0, 'end': 15, 'username': 'Cobham_o...",1026466797577560064,id: 1026466797577560064,2953854791,en,...,2014-04-04T21:26:07.000Z,Video inquiries via the @MailOnline team. Twee...,2427867374,MailOnline Video,2657,1129,27,19167,MailOnlineVideo,False
1,2427867374,author_id: 2427867374,coversation_id: 1001060267860660224,1001060267860660224,2018-08-06T13:44:41.000Z,"[{'start': 0, 'end': 15, 'username': 'Cobham_o...",1026464078661869569,id: 1026464078661869569,2953854791,en,...,2014-04-04T21:26:07.000Z,Video inquiries via the @MailOnline team. Twee...,2427867374,MailOnline Video,2657,1129,27,19167,MailOnlineVideo,False
2,2953854791,author_id: 2953854791,coversation_id: 1001060267860660224,1001060267860660224,2018-08-06T13:52:35.000Z,"[{'start': 0, 'end': 16, 'username': 'MailOnli...",1026466066439053312,id: 1026466066439053312,2427867374,en,...,2014-12-31T22:13:37.000Z,Everything that is Cobham and Oxshott,2953854791,Cobham & Oxshott,2591,173,32,2504,Cobham_oxshott,False
3,2953854791,author_id: 2953854791,coversation_id: 1001060267860660224,1001060267860660224,2018-05-30T17:04:07.000Z,"[{'start': 0, 'end': 14, 'username': 'StevenWo...",1001871892599525379,id: 1001871892599525379,27240829,en,...,2014-12-31T22:13:37.000Z,Everything that is Cobham and Oxshott,2953854791,Cobham & Oxshott,2591,173,32,2504,Cobham_oxshott,False
4,2953854791,author_id: 2953854791,coversation_id: 1001060267860660224,1001060267860660224,2018-05-28T17:22:26.000Z,"[{'start': 0, 'end': 12, 'username': 'LizzieB0...",1001151727104610304,id: 1001151727104610304,330803577,en,...,2014-12-31T22:13:37.000Z,Everything that is Cobham and Oxshott,2953854791,Cobham & Oxshott,2591,173,32,2504,Cobham_oxshott,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,476558551,author_id: 476558551,coversation_id: 998843474953859072,998843474953859072,2018-05-22T08:38:01.000Z,"[{'start': 0, 'end': 10, 'username': 'JunckerE...",998845426538344448,id: 998845426538344448,169426475,en,...,2012-01-28T07:58:34.000Z,NaN,476558551,donald j montgomery,1079,1890,13,43844,donaldjmontgome,False
155,19864496,author_id: 19864496,coversation_id: 998843474953859072,998843474953859072,2018-05-22T08:37:12.000Z,"[{'start': 0, 'end': 10, 'username': 'JunckerE...",998845217468973056,id: 998845217468973056,169426475,en,...,2009-02-01T16:25:18.000Z,"Human rights, current affairs, humanism Do not...",19864496,Owldom #KBF,1400,4531,15,144881,owldom,False
156,3029956305,author_id: 3029956305,coversation_id: 998843474953859072,998843474953859072,2018-05-22T08:35:34.000Z,"[{'start': 0, 'end': 10, 'username': 'JunckerE...",998844809862373376,id: 998844809862373376,169426475,en,...,2015-02-11T10:13:10.000Z,Retweets Are Endorsements,3029956305,Time Pigeon,338,793,7,2654,ThePigeonOfTime,False
157,964191402958688257,author_id: 964191402958688257,coversation_id: 998843474953859072,998843474953859072,2018-05-22T08:33:44.000Z,"[{'start': 0, 'end': 10, 'username': 'JunckerE...",998844347989876736,id: 998844347989876736,169426475,en,...,2018-02-15T17:35:19.000Z,"Brexit - Leave with an FTA, Call EU 27 Summit....",964191402958688257,Don't mention Greece,430,1844,7,27439,Globan999,False


In [11]:
## Merged Tweets with comments (each row is a tweet plus one of its comments, meaning that a tweet with 50 comments
# appears in 50 rows of the table
merged_comments = pd.merge(tweets_table_filtered,all_comments_df,left_on='KOL_Tweet_conversation_id',right_on='Comment_conversation_id' , how="inner")

In [43]:
merged_comments

,KOL_Tweet_Unnamed: 0,KOL_Tweet_Unnamed: 0.1,KOL_Tweet_Unnamed: 0.1.1,KOL_Tweet_author_id,KOL_Tweet_author_id_new,KOL_Tweet_conv_id_new,KOL_Tweet_conversation_id,KOL_Tweet_created_at,KOL_Tweet_entities.mentions,KOL_Tweet_id,...,Comment_users.created_at,Comment_users.description,Comment_users.id,Comment_users.name,Comment_users.public_metrics.followers_count,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified
0,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
1,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
2,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
3,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",765477312,pongodhall,2277,4505,218,263066,pongodhall,False
4,0,26598,188,2797521996,author_id: 2797521996,conv_id: 740159086667411456,740159086667411456,2016-06-07T12:30:52.000Z,NaN,7.401591e+17,...,2016-11-04T16:35:55.000Z,"Literacy, health, processes, nature, space, cu...",794578934511915008,Lorraine O'Mahoney 💙 #stopcancer,5575,6090,54,97942,Lomquiche,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460823,2926,28755,376,103065157,author_id: 103065157,conv_id: 452078384164532224,452078384164532224,2014-04-04T13:41:00.000Z,NaN,4.520784e+17,...,2011-03-21T20:00:20.000Z,"To report an incident call 0800 80 70 60, do n...",269978062,Environment AgencySW,25609,3362,435,18761,EnvAgencySW,True
460824,2926,28755,376,103065157,author_id: 103065157,conv_id: 452078384164532224,452078384164532224,2014-04-04T13:41:00.000Z,NaN,4.520784e+17,...,2009-06-25T23:33:02.000Z,"29, Business Consultant, passion for Cycling a...",50815317,Garry,272,512,9,15398,GarryGT6,False
460825,2926,28755,376,103065157,author_id: 103065157,conv_id: 452078384164532224,452078384164532224,2014-04-04T13:41:00.000Z,NaN,4.520784e+17,...,2014-01-09T10:39:53.000Z,"Northern, done stuff, spot hypocrites, love c...",2283405625,Jack Regan,444,666,11,11016,eddiethehamster,False
460826,2926,28755,376,103065157,author_id: 103065157,conv_id: 452078384164532224,452078384164532224,2014-04-04T13:41:00.000Z,NaN,4.520784e+17,...,2009-02-18T10:30:16.000Z,"Family, Friends,QPR and England the rest is bu...",21184742,adrian clark,775,1304,11,30841,qprade,False


In [92]:
merged_comments.columns

Index(['KOL_Tweet_Unnamed: 0', 'KOL_Tweet_Unnamed: 0.1', 'KOL_Tweet_author_id',
       'KOL_Tweet_conv_id_new', 'KOL_Tweet_conversation_id',
       'KOL_Tweet_created_at', 'KOL_Tweet_entities.mentions', 'KOL_Tweet_id',
       'KOL_Tweet_id_new', 'KOL_Tweet_public_metrics.like_count',
       'KOL_Tweet_public_metrics.quote_count',
       'KOL_Tweet_public_metrics.reply_count',
       'KOL_Tweet_public_metrics.retweet_count', 'KOL_Tweet_referenced_tweets',
       'KOL_Tweet_text', 'KOL_Tweet_users.created_at',
       'KOL_Tweet_users.description', 'KOL_Tweet_users.id',
       'KOL_Tweet_users.name',
       'KOL_Tweet_users.public_metrics.followers_count',
       'KOL_Tweet_users.public_metrics.following_count',
       'KOL_Tweet_users.public_metrics.listed_count',
       'KOL_Tweet_users.public_metrics.tweet_count',
       'KOL_Tweet_users.username', 'KOL_Tweet_users.verified',
       'KOL_Tweet_text_tokens', 'KOL_Tweet_created_at_date',
       'KOL_Tweet_is_in_special_date', 'KOL_Tweet_

In [14]:
final_table = merged_comments[['KOL_Tweet_author_id', 'KOL_Tweet_users.name', 'KOL_Tweet_conversation_id', 'KOL_Tweet_text', 'KOL_Tweet_text_tokens', 
'Comment_author_id', 'Comment_text',
       'Comment_users.created_at', 'Comment_users.description',
'Comment_users.public_metrics.following_count',
       'Comment_users.public_metrics.listed_count',
       'Comment_users.public_metrics.tweet_count', 'Comment_users.username',
       'Comment_users.verified', "Comment_lang"]]

#final_table['KOL ID'] = final_table['KOL ID'].apply(lambda x: x.replace("author_id: ", ""))
#final_table['Commenter ID'] = final_table['Commenter ID'].apply(lambda x: x.replace("author_id: ", ""))

final_table

,KOL_Tweet_author_id,KOL_Tweet_users.name,KOL_Tweet_conversation_id,KOL_Tweet_text,KOL_Tweet_text_tokens,Comment_author_id,Comment_text,Comment_users.created_at,Comment_users.description,Comment_users.public_metrics.following_count,Comment_users.public_metrics.listed_count,Comment_users.public_metrics.tweet_count,Comment_users.username,Comment_users.verified,Comment_lang
0,2797521996,David Davis,740159086667411456,(2/5) The EU is very slow on trade deals. Took...,"['eu', 'slow', 'trade_deals', 'took', 'eu', 'y...",765477312,@Joe90kerrByrd @DavidDavisMP We don't want to ...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",4505,218,263066,pongodhall,False,en
1,2797521996,David Davis,740159086667411456,(2/5) The EU is very slow on trade deals. Took...,"['eu', 'slow', 'trade_deals', 'took', 'eu', 'y...",765477312,@aaaajjjj14 @DavidDavisMP @A50Challenge They d...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",4505,218,263066,pongodhall,False,en
2,2797521996,David Davis,740159086667411456,(2/5) The EU is very slow on trade deals. Took...,"['eu', 'slow', 'trade_deals', 'took', 'eu', 'y...",765477312,@WilliamDLees @DavidDavisMP @A50Challenge Govt...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",4505,218,263066,pongodhall,False,en
3,2797521996,David Davis,740159086667411456,(2/5) The EU is very slow on trade deals. Took...,"['eu', 'slow', 'trade_deals', 'took', 'eu', 'y...",765477312,@DavidDavisMP @A50Challenge Eu. Fussy about Wo...,2012-08-18T10:34:46.000Z,"interested in current affairs - world over,pla...",4505,218,263066,pongodhall,False,en
4,2797521996,David Davis,740159086667411456,(2/5) The EU is very slow on trade deals. Took...,"['eu', 'slow', 'trade_deals', 'took', 'eu', 'y...",794578934511915008,@steffanrhys3 @DavidDavisMP @LeaveEUOfficial T...,2016-11-04T16:35:55.000Z,"Literacy, health, processes, nature, space, cu...",6090,54,97942,Lomquiche,False,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460823,103065157,David Cameron,452078384164532224,"As promised, dredging is underway here in Some...","['promised', 'dredging', 'underway', 'somerset...",269978062,RT @david_cameron: Everyone here is incredibly...,2011-03-21T20:00:20.000Z,"To report an incident call 0800 80 70 60, do n...",3362,435,18761,EnvAgencySW,True,en
460824,103065157,David Cameron,452078384164532224,"As promised, dredging is underway here in Some...","['promised', 'dredging', 'underway', 'somerset...",50815317,"@ChrisPagey @David_Cameron wow, handsome digge...",2009-06-25T23:33:02.000Z,"29, Business Consultant, passion for Cycling a...",512,9,15398,GarryGT6,False,en
460825,103065157,David Cameron,452078384164532224,"As promised, dredging is underway here in Some...","['promised', 'dredging', 'underway', 'somerset...",2283405625,@David_Cameron Aw! Dave! you're awesome!!,2014-01-09T10:39:53.000Z,"Northern, done stuff, spot hypocrites, love c...",666,11,11016,eddiethehamster,False,en
460826,103065157,David Cameron,452078384164532224,"As promised, dredging is underway here in Some...","['promised', 'dredging', 'underway', 'somerset...",21184742,@David_Cameron Again 25 years too late but the...,2009-02-18T10:30:16.000Z,"Family, Friends,QPR and England the rest is bu...",1304,11,30841,qprade,False,en


## Sentiment analysis

In [15]:
#! pip install nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\practikum_team\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [16]:
def nltk_sentiment(row, sentiment_intensity_analyzer, column_text_name = "text"):
    import numpy as np, pandas as pd
    sentiment_names = ['negative', 'neutral', 'positive']
    nltk_sentiment = sentiment_intensity_analyzer.polarity_scores(row[column_text_name])
    
    #nltk_sentiment['nltk_verdict'] = sentiment_names[np.argmax([nltk_sentiment[s] for s in ['neg','neu','pos']])]
    if nltk_sentiment['neu']>0.95:
        nltk_sentiment['nltk_verdict'] = 'neutral'
    elif nltk_sentiment['neg'] < nltk_sentiment['pos']:
        nltk_sentiment['nltk_verdict'] = 'positive'
    else:
        nltk_sentiment['nltk_verdict'] = 'negative'
    #nltk_sentiment['ground_truth'] = row['airline_sentiment']
    #nltk_sentiment['ground_truth_value'] = {'negative': -1, 'neutral':0, 'positive':1 }[nltk_sentiment['ground_truth']]
    return pd.Series(nltk_sentiment)

In [17]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sentiment_intensity_analyzer = SentimentIntensityAnalyzer()

nltk_sentiment_for_KOL_Tweet_text = final_table.apply(nltk_sentiment, sentiment_intensity_analyzer=sentiment_intensity_analyzer, column_text_name = "KOL_Tweet_text"  ,axis=1)
nltk_sentiment_for_Comment_text = final_table.apply(nltk_sentiment, sentiment_intensity_analyzer=sentiment_intensity_analyzer, column_text_name = "Comment_text"  ,axis=1)

##### Adding the sentiment analysis columns
final_table["KOL_Tweet_negative"] = nltk_sentiment_for_KOL_Tweet_text.neg
final_table["KOL_Tweet_neutural"] = nltk_sentiment_for_KOL_Tweet_text.neu
final_table["KOL_Tweet_positive"] = nltk_sentiment_for_KOL_Tweet_text.pos
final_table["KOL_Tweet_compound"] = nltk_sentiment_for_KOL_Tweet_text.compound

final_table["Comment_text_negative"] = nltk_sentiment_for_Comment_text.neg
final_table["Comment_text_neutural"] = nltk_sentiment_for_Comment_text.neu
final_table["Comment_text_positive"] = nltk_sentiment_for_Comment_text.pos
final_table["Comment_text_compound"] = nltk_sentiment_for_Comment_text.compound

#### This columns contain the final "verdict" of the NLTK Model
final_table["KOL_Tweet_nltk_verdict"] = nltk_sentiment_for_KOL_Tweet_text.nltk_verdict
final_table["Comment_text_nltk_verdict"] = nltk_sentiment_for_Comment_text.nltk_verdict

In [5]:
final_table.head()

NameError: name 'final_table' is not defined

In [ ]:
final_table.to_excel('final_table_with_comments.xlsx')